In [39]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

llm = ChatOpenAI(temperature=0.1)

cache_dir = LocalFileStore("./.cache/")

splitter = CharacterTextSplitter.from_tiktoken_encoder(
  separator="\n",
  chunk_size=600,
  chunk_overlap=100,
)

loader = UnstructuredFileLoader("./files/chapter_one.txt")
docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
  embeddings, cache_dir
)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriever = vectorstore.as_retriever()

map_doc_prompt = ChatPromptTemplate.from_messages(
  [
    (
      "system",
      """
      Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
      -------
      {context}
      """,
    ),
    ("human", "{question}"),
  ]
)

map_doc_chain = map_doc_prompt | llm

def map_docs(inputs):
  documents = inputs['documents']
  question = inputs['question']
  # results = []
  # for document in documents:
  #   result = map_doc_chain.invoke({
  #     "context": document.page_content,
  #     "question": question
  #   }).content
  #   results.append(result)
  # print(results)
  # return "\n\n".join(results)
  return "\n\n".join(map_doc_chain.invoke({
      "context": doc.page_content,
      "question": question
    }).content for doc in documents)

map_chain = {"documents": retriever, "question" : RunnablePassthrough()} | RunnableLambda(map_docs)

final_prompt = ChatPromptTemplate.from_messages(
  [
    (
        "system",
        """
        Given the following extracted parts of a long document and a question, create a final answer. 
        If you don't know the answer, just say that you don't know. Don't try to make up an answer.
        ------
        {context}
        """,
    ),
    ("human", "{question}"),
  ]
)

chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

# map reduced type chain


In [40]:
chain.invoke("Describe Victory Mansions.")

AIMessage(content='Victory Mansions is a building located in London, specifically in Airstrip One, which is the chief city of Oceania. It is described as a grimy landscape with rotting nineteenth-century houses. The houses are in a state of disrepair, with their sides supported by timber, windows patched with cardboard, and roofs made of corrugated iron. The garden walls are sagging in all directions. There are also bombed sites with rubble and plaster dust in the air, as well as sordid colonies of wooden dwellings resembling chicken-houses. The specific appearance of Victory Mansions is not described in detail, but it is mentioned that from its roof, one can see the other three buildings that house the Ministries of Truth, Peace, Love, and Plenty.')

In [41]:
chain.invoke("Where does Winston go to work?")

AIMessage(content='Winston goes to work at the Ministry of Truth.')